![TQ42%20Banner.png](attachment:TQ42%20Banner.png)

# Welcome to TQ42

## Example: Using Custom Quantum Layers

## ✨ Introduction 
This notebook demonstrates how to create and use a custom quantum layer within a classical machine learning model using the TQ42 SDK. For more details, please see the [developer documentation](https://docs.tq42.com/en/latest/index.html).

We'll tackle a regression problem with the Boston House Price dataset, predicting house prices based on property and neighborhood features. 

## 🎯 Objective

Applying a custom quantum layer within a classical architecture involves:
1. Preparing and connecting a training dataset
2. Designing a custom quantum layer by creating and ordering the gates available in the TQnet library
3. Integrating the custom quantum layer into a classical neural network model
4. Creating and monitoring an experiment run to train the model using the data

# 1. Import the TQ42 API

In [12]:
# Import packages

from google.protobuf.json_format import MessageToDict
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.dataset import list_all
from tq42.dataset import Dataset


from tq42.algorithm import (
    TrainDataProcessingParametersProto,
    OptimProto,
    LossFuncProto,
    DatasetStorageInfoProto,
    GenericMLTrainMetadataProto,
    GenericMLTrainParametersProto,
    Layer,
    ClassicalDenseLayer,
    MLModelType,
    TrainModelInfoProto,
    MLTrainInputsProto,
    AlgorithmProto,
    MeasureProto,
    CustomQuantumLayer,
    CnotGate,
    HadamardGate,
    VariationalGate,
    EncodingGate,
    MeasurementGate,
    Gate
)

from ipywidgets import interact, interactive, fixed, interact_manual

# 2. Create a client

In [13]:
# Connect to the cloud service
# https://terraquantum.io

with TQ42Client() as client:
    client.login()

{"device_code":"-jlj7aYm-iq34HpVvguwPKzk","user_code":"GDMB-ZQDL","verification_uri":"https://auth.terraquantum.io/activate","expires_in":900,"interval":5,"verification_uri_complete":"https://auth.terraquantum.io/activate?user_code=GDMB-ZQDL"}
If a browser does not open, please access this URL: https://auth.terraquantum.io/activate?user_code=GDMB-ZQDL to login. Also check the code: GDMB-ZQDL
Authentication is successful, access token is saved in: keyring.
org:2b040bcb-1697-491f-83d8-9da045c93eaf
proj:347826c3-5b62-4bd7-ad06-cfd11b9e4746


# 3. Select an organization, project and experiment

In [14]:
# List the organizations available to you and select one

with TQ42Client() as client: 
    org_list = list_all_organizations(client=client)
    
org=None
def f(x):
    global org
    for o in org_list:
        if o.data.name == x:
            org = o
            print(f"Using organization {x}, {org.data.id}")
            return
        
interact(f, x=[o.data.name for o in org_list]);

interactive(children=(Dropdown(description='x', options=('Terra Quantum AG',), value='Terra Quantum AG'), Outp…

In [15]:
# List the projects within that organization and select one
   
with TQ42Client() as client: 
    proj_list = list_all_projects(client=client, organization_id=org.id)
    
proj=None
def f(x):
    global proj
    for p in proj_list:
        if p.data.name == x:
            proj = p
            print(f"Using project {x}, {proj.data.id}")
            return
        
interact(f, x=[p.data.name for p in proj_list]);

interactive(children=(Dropdown(description='x', options=('FRIENDLY_NAME', '0117231015', 'd5328ab0-b06c-4a06-84…

In [16]:
# List the experiments within that project and select one

with TQ42Client() as client: 
    exp_list = list_all_experiments(client=client, project_id=proj.id)
    
exp=None
def f(x):
    global exp
    for e in exp_list:
        if e.data.name == x:
            exp = e
            print(f"Using experiment {x}, {exp.data.id}")
            return
        
interact(f, x=[e.data.name for e in exp_list]);

interactive(children=(Dropdown(description='x', options=('MES-TEST2', 'MES-TEST', 'Test1', 'experiment for pre…

# 4. List Datasets Available in the Project

In [19]:
# List datasets in the project you can use to train your model
# You'll provide the id for one of these datasets along with the input and output columns when you train the model below. If you don't know the input/output columns for the dataset, it's best to upload a new one.

with TQ42Client() as client:
    # datasets = list_all(client=client, project_id="ENTER_PROJECT_ID_HERE")
    datasets = list_all(client=client, project_id="347826c3-5b62-4bd7-ad06-cfd11b9e4746")
    print(datasets)

DatasetList: [
  <Dataset Id=818b9926-b542-4b35-b8ee-07fbe08ce655 Name=Output Dataset>
  <Dataset Id=3bf03f15-e6dd-421b-8b5b-da402c1dc27f Name=boston_regression_valid>
  <Dataset Id=079dca20-92e4-4d3a-ad4a-7d58830b1b40 Name=Output Dataset>
  <Dataset Id=88507758-76f4-4e35-bb8a-eef385a3a7ff Name=Output Dataset>
  <Dataset Id=be9768a4-fc55-4667-a9de-52d1515667df Name=Output Dataset>
  <Dataset Id=2b78ddd2-ed16-4fea-936e-64f5e092a63c Name=ll_boston_regression>
  <Dataset Id=815114ed-00fa-4688-ab86-f9e5d2094538 Name=Output Dataset>
  <Dataset Id=513e9b31-45c0-423e-a524-ea262cb3c297 Name=Output Dataset>
  <Dataset Id=0fa8fe45-6e7e-48e4-8ee2-513505c136e0 Name=Output Dataset>
  <Dataset Id=bc2fd1fc-c58b-4a07-a040-2469046be9fe Name=Dataset Name Test1>
  <Dataset Id=b4ac728b-47b0-49d6-90e1-be0fe002191d Name=Dataset Name Test1>
  <Dataset Id=207fbadf-0b13-4b90-b6db-54525d07f437 Name=Output Dataset>
  <Dataset Id=37ebe265-506c-4126-917c-4d7c9c2b7a7f Name=Output Dataset>
  <Dataset Id=3d652ec8-6e3

# 5. (Optional) Upload a Dataset

In [ ]:
# If you do not already have a dataset in that project, upload a dataset from a cloud storage bucket
# Note: The URL format is gs://[id_of_bucket].

# from tq42.client import TQ42Client
# from tq42.dataset import Dataset, DatasetSensitivityProto

# with TQ42Client() as client:
#     dataset = Dataset.create(
#         client=client,
#         name="<NAME_OF_THE_NEW_DATASET>",
#         description="<DESCRIPTION_OF_THE_NEW_DATASET>",
#         url="gs://<THIS_IS_YOUR_BUCKET_URL>",
#         sensitivity=DatasetSensitivityProto.SENSITIVE,
#         project_id="<PROJECT_ID>",
#     )
#     print(dataset.id)

# 6. Build and Train the Custom-Hybrid Network

Verify your experiment run details

In [20]:
print(f"Running experiment within: Org {org.data.name, org.id}, Proj {proj.data.name, proj.id} and Exp {exp.data.name, exp.id}`")

Running experiment within: Org ('Terra Quantum AG', '2b040bcb-1697-491f-83d8-9da045c93eaf'), Proj ('FRIENDLY_NAME', '347826c3-5b62-4bd7-ad06-cfd11b9e4746') and Exp ('MES-TEST2', '166d459c-3488-4651-8240-cc5a61cd0917')`


In [21]:
# Create and design the Custom Quantum Layer. 
# This is an example sequence, however this sequence is customizable

custom_quantum_layer_msg = CustomQuantumLayer(
    num_qubits=2,
    gates=[
        # Apply Hadamard gates to create superposition
        Gate(hadamard=HadamardGate(wire=0)),
        Gate(hadamard=HadamardGate(wire=1)),
        # Apply a variational gate for parameter optimization
        Gate(variational=VariationalGate(wire=0, rotation=MeasureProto.X)),
        Gate(
            # Encode classical data into the quantum circuit
            encoding=EncodingGate(wire=1, rotation=MeasureProto.Y, feature_id=0) # Encode classical feature 0 as Y rotation on qubit 1
        ),
        # Apply entanglement between qubits
        Gate(cnot=CnotGate(wire1=0, wire2=1)), # CNOT gate with control qubit 0 and target qubit 1
        # Apply another variational gate
        Gate(variational=VariationalGate(wire=1, rotation=MeasureProto.X)),
        # Measure the qubits
        Gate(measurement=MeasurementGate(wire=0, pauli=MeasureProto.X)),
        Gate(measurement=MeasurementGate(wire=1, pauli=MeasureProto.X)),
    ],
)
# Create a classical neural network and integrate the CQ layer
env_msg = GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        model_type=MLModelType.MLP,
        # Add and customize layers here
        layers=[
            Layer(custom_quantum_layer=custom_quantum_layer_msg),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=True)),
        ],
        # WHY ONLY 1 EPOCH? !!!!!!!!!!!!!!!!         !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!         !!!!!!!!!!!!!!!!!!!!!!!!!!
        num_epochs=1,
        k_fold=1,
        batch_size=128,
        learning_rate=0.01,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MAE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model
            name="local_test",
            # Add a brief description to help users understand the purpose or functionality of this trained model
            description="a_description",
        ),
        # Specify the input and output columns for your dataset
        data_processing_parameters=TrainDataProcessingParametersProto(
            input_columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            output_columns=[13]
            # input_columns=[0, 1, 2, 3], output_columns=[4], timestamp_columns=[]
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42
        # data=DatasetStorageInfoProto(storage_id="STORAGE ID")
        data=DatasetStorageInfoProto(storage_id="3bf03f15-e6dd-421b-8b5b-da402c1dc27f")
    ),
)

In [ ]:
# Train the neural network containing the new custom quantum layer

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN,
    # Provide the specific experiement ID your are running on TQ42
    # experiment_id="b2004690-2a4b-479e-bafe-13a475bb0d69",
    experiment_id=exp.id,
    compute=HardwareProto.SMALL,
    parameters=MessageToDict(env_msg, preserving_proto_field_name=True)
)

print(run.data)

In [ ]:
# Poll for the results

result = run.poll()

print(result.data)

### Export Results

In [ ]:
# Export the trained dataset using the storage_id of the inferred_evaluation_data in the results above

# dataset = Dataset(client=client, id="<YOUR_DATASET_ID>")
dataset = Dataset(client=client, id="1023200d-6d36-45ea-b66d-afbcc192eabc")
print(dataset)
# exported_files = dataset.export(directory_path="<YOUR_EXPORT_PATH>")
exported_files = dataset.export(directory_path="/Users/anyaschukin/Coding_Projects/data_samples/Quantum")
print(exported_files)